In [1]:
from flask import jsonify
from flask import Flask
from flask import json
from flask import request
from flask_cors import CORS
from datetime import date
import mysql.connector
import json
from datetime import datetime
import pandas as pd

db_config = {
    'host': '156.67.213.247',
    'user': 'n1569631_admin',
    'password': 'Ohno210500!',
    'database': 'n1569631_pmr_live_UITR24'
}

In [78]:
# Membuat koneksi ke database
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor(dictionary=True)

query = "SELECT result.BIB,result.NAME,result.GENDER,result.CATEGORY,result.FINISHTIME,result.PACE,result.STATUS,result.RACENAME ,cp1.finishtime AS 'CP1' ,cp2.finishtime AS 'CP2' ,cp3.finishtime AS 'CP3' from result LEFT JOIN cp1 ON result.BIB = cp1.bib LEFT JOIN cp2 ON result.BIB = cp2.bib LEFT JOIN cp3 ON result.BIB = cp3.bib"
cursor.execute(query)

# Mengambil semua hasil query
data = cursor.fetchall()

# Menutup kursor dan koneksi
cursor.close()
connection.close()
df = pd.DataFrame(data)

In [77]:
df = df[~df['CATEGORY'].isin(['DNF', 'DNS'])]
df['DETAIL CATEGORY'] = df['RACENAME']+" "+df['CATEGORY']+" "+df['GENDER']
unique_race = df['DETAIL CATEGORY'].unique()

results = {}
# Isi dictionary menggunakan loop for
for race in unique_race:
    df2 = df[df['DETAIL CATEGORY'] == race].copy()
    df2.loc[:, 'FINISHTIME'] = pd.to_datetime(df2['FINISHTIME'], format='%H:%M:%S.%f').dt.strftime('%H:%M:%S.%f')
    df2.loc[:, 'FINISHTIME'] = df2['FINISHTIME'].str[:-4]
    df2 = df2.sort_values(by='FINISHTIME', ascending=True)
    df2.head(10)
    df_results = df2.to_json(orient='records')
    key = f"{race}"
    value = df_results
    results[key] = value

results

{'UITR2480K OPEN MALE': '[{"BIB":"8006","NAME":"ARIEF WISMOYONO","GENDER":"MALE","CATEGORY":"OPEN","FINISHTIME":"16:34:28.38","PACE":"12:26","STATUS":"FINISHER","RACENAME":"UITR2480K","CP1":" 01:42:04.64","CP2":null,"CP3":null,"DETAIL CATEGORY":"UITR2480K OPEN MALE"},{"BIB":"8025","NAME":"SOBARI HERDIANA","GENDER":"MALE","CATEGORY":"OPEN","FINISHTIME":"17:52:44.97","PACE":"13:25","STATUS":"FINISHER","RACENAME":"UITR2480K","CP1":" 11:43:03.18","CP2":null,"CP3":null,"DETAIL CATEGORY":"UITR2480K OPEN MALE"},{"BIB":"8001","NAME":"ABDUL AZIZ DERMAWAN","GENDER":"MALE","CATEGORY":"OPEN","FINISHTIME":"19:10:04.75","PACE":"14:23","STATUS":"FINISHER","RACENAME":"UITR2480K","CP1":" 01:49:56.10","CP2":null,"CP3":null,"DETAIL CATEGORY":"UITR2480K OPEN MALE"},{"BIB":"8024","NAME":"SIMON PIETER WIBISONO GAE","GENDER":"MALE","CATEGORY":"OPEN","FINISHTIME":"20:42:41.09","PACE":"15:32","STATUS":"FINISHER","RACENAME":"UITR2480K","CP1":" 01:51:27.23","CP2":null,"CP3":null,"DETAIL CATEGORY":"UITR2480K OPEN